# Adquiriendo token

## Importar Librerías

In [59]:
import pandas as pd
import requests
import json
import os #manipular archivos del sistema
import base64 #codificar informacion para enviarla a la url

#Variables de entorno donde alamcenamos cables secretas
from dotenv import load_dotenv
load_dotenv("Credentials.env") #Para evitar que se comparta dicha información si queremos compartir el código

True

## Extraemos token

In [61]:
#almacenamos variables de entorno en local
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [63]:
#combinemos la informacion para solicitar token o llave de acceso a spotify y luego si descargar información
def get_token():
    #concatenamos informacion
    auth_str = client_id + ":" + client_secret
    #Ahora codificamos la cadena de autentificacion en bites
    auth_bytes = auth_str.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes),"utf-8") #convertimos a base64

    #Llamamos al servicio de autentificación de spotify
    url = "https://accounts.spotify.com/api/token"
    #Ahora le pasamos las cabeceras
    headers = {
        "Authorization": "Basic " + auth_base64, #Pasamos la autorización
        #El encabezado debe tener un espacio entre "Basic" y auth_base64.
        "Content-Type": "application/x-www-form-urlencoded"}
    # luego la info y datos de solicitud del metodo post de http, en este caso indicamos una consesion de tipo gr type
    data = {"grant_type":"client_credentials"}
    #Por ultimo hacemos la solicitud http
    result = requests.post(url=url, headers=headers, data=data)
    json_result = json.loads(result.content) #Ya que nos arroja un json

    token = json_result['access_token']

    return token

#Hay que construir un prestring con la autorizacion + token = clave completa, toke: clave parcial 
def get_auth_token(token):
    #Tetornamos diccionarion con encabezado de autorizacion utilizando el token
    return {"Authorization": 'Bearer ' + token}

In [65]:
token = get_token()
token

'BQDTM1MYkRv1Gjocy-yoiD2aX7PUvDUqnr3hMk2yVy3yKSDa4W7XXeppLjr5g4YkMTv8BggvmzE0eORLZGeTSoZge1-BEU4j1VM89ayK1R3Hzhx8X1lm7MfgnaekHls3Z942VOK4R00'

# Carga de la data

In [67]:
data = pd.read_json('C:/Users/USER/OneDrive - Universidad Nacional de Colombia/Escritorio/Proyecto Spotify/2023-2024.json',encoding='utf-8')

# EDA (Exploratory Data Analysis)

In [69]:
pd.set_option('display.max_columns', None)
data.head(5)

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,audiobook_title,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2023-01-17T22:31:09Z,android,54960,CO,179.1.129.55,"Paulo Londra: Bzrp Music Sessions, Vol. 23",Bizarrap,"Paulo Londra: Bzrp Music Sessions, Vol. 23",spotify:track:4F4bzCA8RWoyKQDfeFGolM,None,None,None,NaN,NaN,NaN,NaN,fwdbtn,logout,False,False,False,1673993501,False
1,2023-01-18T02:36:19Z,android,141280,CO,179.1.129.55,Between The Bars,Elliott Smith,Either/Or,spotify:track:52Bg6oaos7twR7IUtEpqcE,None,None,None,NaN,NaN,NaN,NaN,clickrow,endplay,False,True,False,1674009237,False
2,2023-01-18T02:38:39Z,android,140626,CO,179.1.129.55,Say Yes,Elliott Smith,Either/Or,spotify:track:7zVfEu1dEvRw4EkWoqrBkx,None,None,None,NaN,NaN,NaN,NaN,trackdone,trackdone,False,False,False,1674009379,False
3,2023-01-18T02:39:47Z,android,65418,CO,179.1.129.55,Waiting for the Moon to Rise,Belle and Sebastian,"Fold Your Hands Child, You Walk Like a Peasant",spotify:track:1wtGtVfMHjeNfNLeNFVMK6,None,None,None,NaN,NaN,NaN,NaN,trackdone,unexpected-exit-while-paused,False,False,False,1674009520,False
4,2023-02-01T12:50:27Z,android,174000,CO,45.238.183.125,Siendo Sincero,Los Del Limit,Beyond The Limit,spotify:track:5xyI9sNt8spjqroRSEbXEs,None,None,None,NaN,NaN,NaN,NaN,playbtn,trackdone,True,False,False,1675255644,False


In [71]:
#Creamos un resumen utilizando funciones nativas de pandas
def summarise(data):
  summary = data.describe(include='all').T #Descripción general Si no colocamos include solo se asignará las numéricas
  summary['Type'] = data.dtypes #Tipo de dato
  summary['Unique Values'] = data.nunique() #Cantidad de Valores únicos
  summary['Null Values']= data.isnull().sum()
  summary['Examples'] = data.apply(lambda col: col.dropna().unique()[:3]) #Ejemplos de valores
  summary['Duplicates'] = data.apply(lambda col: col.duplicated().sum())

  #Reorganizar columnas para mejorar visualización
  summary = summary[['Type', 'Null Values','Unique Values', 'Examples','Duplicates']]
  return summary

In [25]:
print(data.shape)

(16044, 23)


In [73]:
summarise(data)

,Type,Null Values,Unique Values,Examples,Duplicates
ts,object,0,15919,"[2023-01-17T22:31:09Z, 2023-01-18T02:36:19Z, 2...",125
platform,object,0,2,"[android, windows]",16042
ms_played,int64,0,9379,"[54960, 141280, 140626]",6665
conn_country,object,0,1,[CO],16043
ip_addr,object,0,277,"[179.1.129.55, 45.238.183.125, 191.91.3.143]",15767
master_metadata_track_name,object,429,2480,"[Paulo Londra: Bzrp Music Sessions, Vol. 23, B...",13563
master_metadata_album_artist_name,object,429,1415,"[Bizarrap, Elliott Smith, Belle and Sebastian]",14628
master_metadata_album_album_name,object,429,2132,"[Paulo Londra: Bzrp Music Sessions, Vol. 23, E...",13911
spotify_track_uri,object,429,2579,"[spotify:track:4F4bzCA8RWoyKQDfeFGolM, spotify...",13464
episode_name,object,15615,197,"[Practice Your English Listening, English List...",15846


In [33]:
duplicated_rows = df[df['ts'].isin(duplicated_ts.index)]
duplicated_rows

,ts,platform,ms_played,conn_country,ip_addr,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,...,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
535,2023-02-25 01:27:34,android,1524,CO,191.93.155.216,Mil Horas,Los Abuelos De La Nada,1982-1987 - Himnos Del Corazón,spotify:track:7JEnT0uFWcwZqNZWv7hC9N,None,...,NaN,NaN,NaN,fwdbtn,fwdbtn,True,True,False,1677288453,False
536,2023-02-25 01:27:34,android,1404,CO,191.93.155.216,De Música Ligera - Remasterizado 2007,Soda Stereo,Canción Animal,spotify:track:2lpIh6Gr6HYjg1CFBaucS5,None,...,NaN,NaN,NaN,fwdbtn,fwdbtn,True,True,False,1677288451,False
1192,2023-10-02 16:46:03,android,0,CO,181.51.32.24,None,None,None,None,Sueño: ¡A contar ovejas!,...,NaN,NaN,NaN,fwdbtn,backbtn,False,True,False,1696265164,False
1193,2023-10-02 16:46:03,android,2412,CO,181.51.32.24,None,None,None,None,Astrología ¿los géminis son lo peor?,...,NaN,NaN,NaN,fwdbtn,fwdbtn,False,True,False,1696265161,False
1195,2023-10-02 16:46:11,android,1150,CO,181.51.32.24,None,None,None,None,Sueño: ¡A contar ovejas!,...,NaN,NaN,NaN,fwdbtn,backbtn,False,True,False,1696265164,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15281,2024-11-25 14:21:25,android,1203,CO,2800:e2:1c00:673:b5a5:2f6d:fc76:9a26,Haste Que El Cuerpo Aguante,Mägo de Oz,Finisterra,spotify:track:5Q2uU5NGcS0mKwmgF28kRZ,None,...,NaN,NaN,NaN,fwdbtn,fwdbtn,True,True,False,1732544483,False
15283,2024-11-25 14:21:28,android,1227,CO,2800:e2:1c00:673:b5a5:2f6d:fc76:9a26,Jogadinha do Paqueta,Mc Rf,Jogadinha do Paqueta,spotify:track:3BCUqGVPvirkRa6g6ipBaF,None,...,NaN,NaN,NaN,fwdbtn,fwdbtn,True,True,False,1732544486,False
15284,2024-11-25 14:21:28,android,637,CO,2800:e2:1c00:673:b5a5:2f6d:fc76:9a26,Perfecta,Miranda!,El Templo del Pop,spotify:track:09QfIIP4NUx4A3thmovb2o,None,...,NaN,NaN,NaN,fwdbtn,fwdbtn,True,True,False,1732544488,False
15914,2024-12-06 21:20:44,android,1834,CO,2800:e2:1c00:673:d1c5:d718:3656:a060,Tu Falta De Querer,Mon Laferte,Mon Laferte,spotify:track:4skuEIloXWuxxgekKupkEH,None,...,NaN,NaN,NaN,unknown,endplay,False,True,False,1733520041,False


In [53]:
data[data['master_metadata_track_name']=='Between The Bars']

,ts,platform,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,episode_show_name,reason_start,reason_end,shuffle,skipped,offline,Real_duration
1,2023-01-18,android,141280,Between The Bars,Elliott Smith,Either/Or,None,clickrow,endplay,False,True,False,141280.0
53,2023-02-02,android,141280,Between The Bars,Elliott Smith,Either/Or,None,clickrow,trackdone,True,False,False,141280.0
158,2023-02-18,android,0,Between The Bars,Elliott Smith,Either/Or,None,clickrow,endplay,True,True,False,141280.0
201,2023-02-19,android,141280,Between The Bars,Elliott Smith,Either/Or,None,clickrow,endplay,True,True,False,141280.0
405,2023-02-21,android,77590,Between The Bars,Elliott Smith,Either/Or,None,clickrow,endplay,True,True,False,141280.0
3007,2024-04-22,android,207458,Between The Bars,Elliott Smith,Either/Or,None,clickrow,endplay,True,True,False,141280.0
3394,2024-04-25,android,16058,Between The Bars,Elliott Smith,Either/Or,None,clickrow,endplay,True,True,False,141280.0
3397,2024-04-25,android,141280,Between The Bars,Elliott Smith,Either/Or,None,clickrow,endplay,True,True,False,141280.0


# PIPELINE ETL - Única celda

In [75]:
def transformaciones(data):

    
    #SOLTAMOS COLUMNAS INNECESARIAS
    data.drop(columns=["incognito_mode", "offline_timestamp", "audiobook_chapter_title", 
                            "audiobook_title","audiobook_uri", "audiobook_chapter_uri", "spotify_episode_uri", 
                             "ip_addr", "conn_country","episode_name",'episode_show_name'],inplace=True)

    #RENOMBRAMOS COLUMNAS
    data.rename(columns={'ts':'date','ms_played': 'time_listened', 'master_metadata_track_name':'track_name',
                    'master_metadata_album_artist_name':'artist_name','master_metadata_album_album_name':'album_name',
                       }, inplace=True)

    #ELIMINACIÓN DE DUPLICADOS VARIABLE CLAVE
    data.drop_duplicates(subset='date', keep='first', inplace=True)


    #TRANSFORMACIONES DE COLUMNAS

    # CATEGORIZAR reason_start and reason_end
   
    categorias_start = {
            'clickrow': 'Manual',
            'fwdbtn': 'Siguiente',
            'trackdone': 'Automático',
            'playbtn': 'Manual',
            'appload': 'Automático',
            'backbtn': 'Siguiente',
            'resumed': 'Automático',
            'trackerror': 'Automático'
        }
        
    data['reason_start'] = data['reason_start'].map(categorias_start).fillna('Otro')

    categorias_end = {
            'logout': 'Automático',
            'endplay': 'Manual',
            'trackdone': 'Automático',
            'unexpected-exit-while-paused': 'Automático',
            'fwdbtn': 'Manual',
            'backbtn': 'Manual',
            'unexpected-exit': 'Automático'
            }

    data['reason_end'] = data['reason_end'].map(categorias_end).fillna('Otro')

    #----------------------------------------------------------------------------------------------------

    #EXTRACCIÓN DURACIÓN REAL PARA POSTERIORMENTE AGREGAR UNA NUEVA COLUMNA DE CATEGORIZACIÓN DE SKIPPED
    
    data['track_id'] = data['spotify_track_uri'].apply(lambda x: x.split(':')[-1] if pd.notna(x) else None)
    
    track_ids = data['track_id'].dropna().unique().tolist()

    # Dividimos en batches de máximo 50 (límite de la API de Spotify)
    all_durations = {}
    for i in range(0, len(track_ids), 50):
        batch = track_ids[i:i+50]
        url = f"https://api.spotify.com/v1/tracks?ids={','.join(batch)}"
        headers = get_auth_token(token)
        response = requests.get(url, headers=headers)
    
        if response.status_code == 200:
            tracks_data = response.json()
            for track in tracks_data['tracks']:
                if track:
                    all_durations[track['id']] = track['duration_ms']
        else:
            print(f"Error {response.status_code} en el batch {batch}")

    # Mapeamos las duraciones al DataFrame
    data['actual_duration'] = data['track_id'].map(all_durations)

    #------------------------------------------------------------------------------------------------------

    #CATEGORIZAMOS PIPELINE
    
    def skipped_type(time_listened,actual_duration,skipped):
        if pd.isna(actual_duration):
            return None
        if not skipped or time_listened >= 0.8 * actual_duration:
            return 'enjoyed'
        elif time_listened <= 0.2 * actual_duration:
            return 'quick skipped'
        else:
            return 'incomplete listen'
            
    data['skipped_category']=data.apply(lambda row: 
    skipped_type(row['time_listened'], row['actual_duration'], row['skipped']), axis=1)


    #TRANSFORMACIONES FINALES - TIPO Y FORMATO
    
    #data['time_listened'] = data['time_listened'].apply(lambda x: f"{x // 60000}:{(x % 60000) // 1000:02}")
    data.drop(columns=["actual_duration",'track_id','spotify_track_uri'],inplace=True)
    data['date'] = pd.to_datetime(data['date']).dt.tz_convert(None)


    data.to_excel('C:/Users/USER/OneDrive - Universidad Nacional de Colombia/Escritorio/Proyecto Spotify/archivo_nuevo.xlsx',index=False)
    data = 'C:/Users/USER/OneDrive - Universidad Nacional de Colombia/Escritorio/Proyecto Spotify/archivo_nuevo.xlsx'
    

    #-------------------------------------------------------------------------------------------------------


    #Código para Carga Incremental con data 1:

    
    # Ruta de los archivos Excel
    archivo_completo = 'C:/Users/USER/OneDrive - Universidad Nacional de Colombia/Escritorio/Proyecto Spotify/carga_incremental.xlsx'  # Archivo donde se guardará la carga completa
    
    # Función para carga incremental con Excel
    def carga_incremental_excel(data, archivo_completo_path):
        # Leer el archivo de datos nuevos
        df_nuevos = pd.read_excel(data)
        
        # Verificar si el archivo completo ya existe
        if os.path.exists(archivo_completo_path):
            # Leer el archivo completo (si existe) y combinarlo con los nuevos datos
            df_completo = pd.read_excel(archivo_completo_path)
            # Concatenar los datos y eliminar duplicados basados en 'id'
            df_actualizado = pd.concat([df_completo, df_nuevos]).drop_duplicates(subset=['date'], keep='last') #eliminar repetidos id, conservar el último
            print(f"Datos existentes combinados con los nuevos. Total registros después de la carga: {len(df_actualizado)}")
        else:
            # Si el archivo no existe, usar solo los datos nuevos
            df_actualizado = df_nuevos
            print(f"Archivo completo no existe. Usando solo los datos nuevos. Total registros: {len(df_actualizado)}")
        
        # Guardar el archivo actualizado en Excel
        df_actualizado.to_excel(archivo_completo_path, index=False)
        print(f"Carga incremental realizada. Datos guardados en {archivo_completo_path}")
    
    # Llamar a la función con el archivo de datos nuevos 1
    carga_incremental_excel(data, archivo_completo)

    #------------------------------------------------------------------------------------------------------

transformaciones(data)

Archivo completo no existe. Usando solo los datos nuevos. Total registros: 15919
Carga incremental realizada. Datos guardados en C:/Users/USER/OneDrive - Universidad Nacional de Colombia/Escritorio/Proyecto Spotify/carga_incremental.xlsx


In [77]:
# Revisamos nuevamente que la data haya salido bien
data.head()

,date,platform,time_listened,track_name,artist_name,album_name,reason_start,reason_end,shuffle,skipped,offline,skipped_category
0,2023-01-17 22:31:09,android,54960,"Paulo Londra: Bzrp Music Sessions, Vol. 23",Bizarrap,"Paulo Londra: Bzrp Music Sessions, Vol. 23",Siguiente,Automático,False,False,False,enjoyed
1,2023-01-18 02:36:19,android,141280,Between The Bars,Elliott Smith,Either/Or,Manual,Manual,False,True,False,enjoyed
2,2023-01-18 02:38:39,android,140626,Say Yes,Elliott Smith,Either/Or,Automático,Automático,False,False,False,enjoyed
3,2023-01-18 02:39:47,android,65418,Waiting for the Moon to Rise,Belle and Sebastian,"Fold Your Hands Child, You Walk Like a Peasant",Automático,Automático,False,False,False,enjoyed
4,2023-02-01 12:50:27,android,174000,Siendo Sincero,Los Del Limit,Beyond The Limit,Manual,Automático,True,False,False,enjoyed


# ETL Pipeline

## Limpieza de Datos

In [ ]:
#SOLTAMOS COLUMNAS INNECESARIAS
data.drop(columns=["incognito_mode", "offline_timestamp", "audiobook_chapter_title", 
                            "audiobook_title","audiobook_uri", "audiobook_chapter_uri", "spotify_episode_uri", 
                             "ip_addr", "conn_country","episode_name",'episode_show_name'],inplace=True)

#RENOMBRAMOS COLUMNAS
data.rename(columns={'ts':'date','ms_played': 'time_listened', 'master_metadata_track_name':'track_name',
                    'master_metadata_album_artist_name':'artist_name','master_metadata_album_album_name':'album_name'
            }, inplace=True)

#ELIMINACIÓN DE DUPLICADOS VARIABLE CLAVE
data.drop_duplicates(subset='date', keep='first', inplace=True)

## Transformación de Columnas

In [ ]:
categorias_start = {
            'clickrow': 'Manual',
            'fwdbtn': 'Siguiente',
            'trackdone': 'Automático',
            'playbtn': 'Manual',
            'appload': 'Automático',
            'backbtn': 'Siguiente',
            'resumed': 'Automático',
            'trackerror': 'Automático'
                    }
        
data['reason_start'] = data['reason_start'].map(categorias_start).fillna('Otro')

categorias_end = {
            'logout': 'Automático',
            'endplay': 'Manual',
            'trackdone': 'Automático',
            'unexpected-exit-while-paused': 'Automático',
            'fwdbtn': 'Manual',
            'backbtn': 'Manual',
            'unexpected-exit': 'Automático'
                }

data['reason_end'] = data['reason_end'].map(categorias_end).fillna('Otro')

## Extración Externa de data con API

In [ ]:
data['track_id'] = data['spotify_track_uri'].apply(lambda x: x.split(':')[-1] if pd.notna(x) else None)
    
track_ids = data['track_id'].dropna().unique().tolist()

# Dividimos en batches de máximo 50 (límite de la API de Spotify)
all_durations = {}
for i in range(0, len(track_ids), 50):
    batch = track_ids[i:i+50]
    url = f"https://api.spotify.com/v1/tracks?ids={','.join(batch)}"
    headers = get_auth_token(token)
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        tracks_data = response.json()
        for track in tracks_data['tracks']:
            if track:
                all_durations[track['id']] = track['duration_ms']
    else:
        print(f"Error {response.status_code} en el batch {batch}")

# Mapeamos las duraciones al DataFrame
data['actual_duration'] = data['track_id'].map(all_durations)

## Categorización de Columna

In [ ]:
def skipped_type(time_listened,actual_duration,skipped):
    if pd.isna(actual_duration):
        return None
    if not skipped or time_listened >= 0.8 * actual_duration:
        return 'enjoyed'
    elif time_listened <= 0.2 * actual_duration:
        return 'quick skipped'
    else:
        return 'incomplete listen'
            
data['skipped_category']=data.apply(lambda row: 
skipped_type(row['time_listened'], row['actual_duration'], row['skipped']), axis=1)


## TRANSFORMACIONES FINALES - TIPO Y FORMATO

In [ ]:
#data['time_listened'] = data['time_listened'].apply(lambda x: f"{x // 60000}:{(x % 60000) // 1000:02}")
data.drop(columns=["actual_duration",'track_id','spotify_track_uri'],inplace=True)
data['date'] = pd.to_datetime(data['date']).dt.tz_convert(None)


data.to_excel('C:/Users/USER/OneDrive - Universidad Nacional de Colombia/Escritorio/Proyecto Spotify/archivo_nuevo.xlsx',index=False)
data = 'C:/Users/USER/OneDrive - Universidad Nacional de Colombia/Escritorio/Proyecto Spotify/archivo_nuevo.xlsx'

## Carga incremental de la data

In [ ]:
# Ruta de los archivos Excel
archivo_completo = 'carga_incremental.xlsx'  # Archivo donde se guardará la carga completa
    
# Función para carga incremental con Excel
def carga_incremental_excel(data, archivo_completo_path):
    
    # Leer el archivo de datos nuevos
    df_nuevos = pd.read_excel(data)
        
    # Verificar si el archivo completo ya existe
    if os.path.exists(archivo_completo_path):
    
        # Leer el archivo completo (si existe) y combinarlo con los nuevos datos
        df_completo = pd.read_excel(archivo_completo_path)
        
        # Concatenar los datos y eliminar duplicados basados en 'id'
        df_actualizado = pd.concat([df_completo, df_nuevos]).drop_duplicates(subset=['date'], keep='last') #eliminar repetidos id, conservar el último
        
        print(f"Datos existentes combinados con los nuevos. Total registros después de la carga: {len(df_actualizado)}")
    else:
        # Si el archivo no existe, usar solo los datos nuevos
        df_actualizado = df_nuevos
        print(f"Archivo completo no existe. Usando solo los datos nuevos. Total registros: {len(df_actualizado)}")
        
        # Guardar el archivo actualizado en Excel
    df_actualizado.to_excel(archivo_completo_path, index=False)
    print(f"Carga incremental realizada. Datos guardados en {archivo_completo_path}")
    

carga_incremental_excel(data, archivo_completo)
